# 🏃 Half Marathon Time Prediction - Training Pipeline
## Professional ML Pipeline with Digital Ocean Spaces Integration

In [ ]:
# Install required packages
# !pip install boto3 pandas numpy scikit-learn xgboost joblib python-dotenv

In [1]:
import pandas as pd
import numpy as np
import boto3
import joblib
import os
from io import BytesIO, StringIO
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## 🔐 Configuration & Digital Ocean Connection

In [2]:
# Digital Ocean Spaces Configuration
DO_SPACES_KEY = os.getenv('DO_SPACES_KEY', 'your_access_key')
DO_SPACES_SECRET = os.getenv('DO_SPACES_SECRET', 'your_secret_key')
DO_SPACES_REGION = os.getenv('DO_SPACES_REGION', 'fra1')
DO_SPACES_BUCKET = os.getenv('DO_SPACES_BUCKET', 'halfmarathon-ml')
DO_SPACES_ENDPOINT = f'https://{DO_SPACES_REGION}.digitaloceanspaces.com'

# Initialize S3 client (Digital Ocean Spaces is S3-compatible)
s3_client = boto3.client(
    's3',
    region_name=DO_SPACES_REGION,
    endpoint_url=DO_SPACES_ENDPOINT,
    aws_access_key_id=DO_SPACES_KEY,
    aws_secret_access_key=DO_SPACES_SECRET
)

print(f"✅ Connected to Digital Ocean Spaces: {DO_SPACES_BUCKET}")

✅ Connected to Digital Ocean Spaces: half-marathon-app


## 📥 Data Loading from Digital Ocean Spaces

In [3]:
def load_csv_from_spaces(filename):
    """Load CSV file from Digital Ocean Spaces"""
    try:
        obj = s3_client.get_object(Bucket=DO_SPACES_BUCKET, Key=filename)  # ← BEZ data/
        df = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8')), sep=';')
        print(f"✅ Loaded {filename}: {len(df)} rows, {len(df.columns)} columns")
        return df
    except Exception as e:
        print(f"❌ Error loading {filename}: {e}")
        return None

# Load datasets
df_2023 = load_csv_from_spaces('halfmarathon_wroclaw_2023__final.csv')
df_2024 = load_csv_from_spaces('halfmarathon_wroclaw_2024__final.csv')

# Combine datasets
df = pd.concat([df_2023, df_2024], ignore_index=True)
print(f"\n📊 Total combined dataset: {len(df)} rows")
df.head()

✅ Loaded halfmarathon_wroclaw_2023__final.csv: 8950 rows, 27 columns
✅ Loaded halfmarathon_wroclaw_2024__final.csv: 13007 rows, 27 columns

📊 Total combined dataset: 21957 rows


,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
0,1.0,1787,TOMASZ,GRYCKO,NaN,POL,UKS BLIZA WŁADYSŁAWOWO,M,1.0,M30,...,2.926667,00:44:47,1.0,3.106667,01:01:43,1.0,3.386667,0.031400,01:04:59,3.080509
1,2.0,3,ARKADIUSZ,GARDZIELEWSKI,WROCŁAW,POL,ARKADIUSZGARDZIELEWSKI.PL,M,2.0,M30,...,2.983333,00:45:26,2.0,3.143333,01:03:08,2.0,3.540000,0.038000,01:06:23,3.146875
2,3.0,3832,KRZYSZTOF,HADAS,POZNAŃ,POL,NaN,M,3.0,M20,...,3.123333,00:47:34,3.0,3.236667,01:05:09,3.0,3.516667,0.024067,01:08:24,3.242475
3,4.0,416,DAMIAN,DYDUCH,KĘPNO,POL,AZS POLITECHNIKA OPOLSKA,M,4.0,M30,...,3.196667,00:48:49,5.0,3.330000,01:06:54,4.0,3.616667,0.025467,01:10:16,3.330963
4,5.0,8476,KAMIL,MAŃKOWSKI,MIRKÓW,POL,PARKRUN WROCŁAW,M,5.0,M20,...,3.276667,00:49:31,7.0,3.386667,01:07:27,5.0,3.586667,0.023000,01:10:27,3.339654


## 🧹 Data Cleaning & Feature Engineering

In [4]:
def convert_time_to_seconds(time_str):
    """Convert time format HH:MM:SS to seconds"""
    if pd.isnull(time_str) or time_str in ['DNS', 'DNF', '']:
        return None
    try:
        parts = str(time_str).split(':')
        if len(parts) == 3:
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
        return None
    except:
        return None

def calculate_age(birth_year):
    """Calculate age from birth year"""
    current_year = datetime.now().year
    try:
        return current_year - int(birth_year)
    except:
        return None

print("🔄 Starting data cleaning...")

# Convert time columns to seconds
time_columns = ['5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', 'Czas']
for col in time_columns:
    if col in df.columns:
        df[f'{col}_seconds'] = df[col].apply(convert_time_to_seconds)

# Calculate age
if 'Rocznik' in df.columns:
    df['Wiek'] = df['Rocznik'].apply(calculate_age)

# Encode gender
if 'Płeć' in df.columns:
    le_gender = LabelEncoder()
    df['Płeć_encoded'] = le_gender.fit_transform(df['Płeć'].fillna('M'))
    joblib.dump(le_gender, 'gender_encoder.pkl')

# Remove rows with missing target variable
df_clean = df[df['Czas_seconds'].notna()].copy()

# Remove outliers (times > 4 hours or < 1 hour)
df_clean = df_clean[
    (df_clean['Czas_seconds'] >= 3600) & 
    (df_clean['Czas_seconds'] <= 14400)
]

print(f"✅ Data cleaning complete: {len(df_clean)} valid rows")
print(f"📉 Removed {len(df) - len(df_clean)} invalid/outlier rows")

🔄 Starting data cleaning...
✅ Data cleaning complete: 18450 valid rows
📉 Removed 3507 invalid/outlier rows


## 🎯 Feature Selection & Engineering

In [5]:
# Select features for modeling
feature_cols = [
    'Płeć_encoded',
    'Wiek',
    '5 km Czas_seconds',
    '5 km Tempo',
]

# Add optional features if available
optional_features = ['10 km Tempo', '15 km Tempo', 'Tempo Stabilność']
for feat in optional_features:
    if feat in df_clean.columns:
        feature_cols.append(feat)

target_col = 'Czas_seconds'

# Create feature matrix
df_model = df_clean[feature_cols + [target_col]].dropna()

X = df_model[feature_cols]
y = df_model[target_col]

print(f"📊 Feature matrix shape: {X.shape}")
print(f"📋 Features used: {feature_cols}")
print(f"\n📈 Target variable statistics:")
print(f"   Mean: {y.mean()/60:.2f} minutes")
print(f"   Median: {y.median()/60:.2f} minutes")
print(f"   Std: {y.std()/60:.2f} minutes")

📊 Feature matrix shape: (17895, 7)
📋 Features used: ['Płeć_encoded', 'Wiek', '5 km Czas_seconds', '5 km Tempo', '10 km Tempo', '15 km Tempo', 'Tempo Stabilność']

📈 Target variable statistics:
   Mean: 123.19 minutes
   Median: 120.67 minutes
   Std: 20.18 minutes


## 🔀 Train-Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✅ Train set: {X_train.shape[0]} samples")
print(f"✅ Test set: {X_test.shape[0]} samples")

✅ Train set: 14316 samples
✅ Test set: 3579 samples


In [ ]:
# ← NOWE: Zapisz feature_order w metadata
model_metadata['features'] = feature_cols  # ← KLUCZOWE!
model_metadata['feature_types'] = {
    col: str(X_train[col].dtype) for col in feature_cols
}

print(f"✅ Feature order zapisana: {feature_cols}")

## 🤖 Model Training & Hyperparameter Tuning

In [7]:
print("🎯 Training XGBoost model with hyperparameter tuning...\n")

# Define parameter grid for XGBoost
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
}

# Initialize XGBoost
xgb_model = XGBRegressor(
    random_state=42,
    n_jobs=-1,
    objective='reg:squarederror'
)

# Grid search with cross-validation
grid_search = GridSearchCV(
    xgb_model,
    param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\n✅ Best parameters: {grid_search.best_params_}")
print(f"✅ Best CV MAE: {-grid_search.best_score_/60:.2f} minutes")

# Use best model
best_model = grid_search.best_estimator_

🎯 Training XGBoost model with hyperparameter tuning...

Fitting 5 folds for each of 81 candidates, totalling 405 fits

✅ Best parameters: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 300, 'subsample': 0.9}
✅ Best CV MAE: 0.70 minutes


## 📊 Model Evaluation

In [8]:
# Predictions
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Calculate metrics
def evaluate_model(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n📊 {dataset_name} Metrics:")
    print(f"   MAE: {mae/60:.2f} minutes ({mae:.0f} seconds)")
    print(f"   RMSE: {rmse/60:.2f} minutes ({rmse:.0f} seconds)")
    print(f"   R² Score: {r2:.4f}")
    
    return {'mae': mae, 'rmse': rmse, 'r2': r2}

train_metrics = evaluate_model(y_train, y_pred_train, "Training Set")
test_metrics = evaluate_model(y_test, y_pred_test, "Test Set")

# Feature importance
print("\n🎯 Feature Importance:")
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)
print(feature_importance.to_string(index=False))


📊 Training Set Metrics:
   MAE: 0.44 minutes (26 seconds)
   RMSE: 0.58 minutes (35 seconds)
   R² Score: 0.9992

📊 Test Set Metrics:
   MAE: 0.68 minutes (41 seconds)
   RMSE: 1.14 minutes (69 seconds)
   R² Score: 0.9968

🎯 Feature Importance:
          feature  importance
      15 km Tempo    0.867453
      10 km Tempo    0.107561
5 km Czas_seconds    0.015539
 Tempo Stabilność    0.008603
             Wiek    0.000455
     Płeć_encoded    0.000389
       5 km Tempo    0.000000


## 💾 Save Model Locally & Upload to Digital Ocean Spaces

In [9]:
# Create model metadata
model_metadata = {
    'model_type': 'XGBoost',
    'version': datetime.now().strftime('%Y%m%d_%H%M%S'),
    'features': feature_cols,
    'train_samples': len(X_train),
    'test_samples': len(X_test),
    'metrics': {
        'train': train_metrics,
        'test': test_metrics
    },
    'best_params': grid_search.best_params_
}

# Save model locally
model_filename = f"halfmarathon_model_{model_metadata['version']}.pkl"
metadata_filename = f"model_metadata_{model_metadata['version']}.pkl"

joblib.dump(best_model, model_filename)
joblib.dump(model_metadata, metadata_filename)

print(f"✅ Model saved locally: {model_filename}")
print(f"✅ Metadata saved locally: {metadata_filename}")

# Upload to Digital Ocean Spaces
def upload_to_spaces(local_file, spaces_key):
    try:
        s3_client.upload_file(
            local_file,
            DO_SPACES_BUCKET,
            f'models/{spaces_key}',
            ExtraArgs={'ACL': 'private'}
        )
        print(f"✅ Uploaded to Spaces: models/{spaces_key}")
    except Exception as e:
        print(f"❌ Upload error: {e}")

upload_to_spaces(model_filename, model_filename)
upload_to_spaces(metadata_filename, metadata_filename)
upload_to_spaces('gender_encoder.pkl', 'gender_encoder.pkl')

# Also save as "latest" version
upload_to_spaces(model_filename, 'halfmarathon_model_latest.pkl')
upload_to_spaces(metadata_filename, 'model_metadata_latest.pkl')

print("\n🎉 Training pipeline completed successfully!")

✅ Model saved locally: halfmarathon_model_20250929_215634.pkl
✅ Metadata saved locally: model_metadata_20250929_215634.pkl
✅ Uploaded to Spaces: models/halfmarathon_model_20250929_215634.pkl
✅ Uploaded to Spaces: models/model_metadata_20250929_215634.pkl
✅ Uploaded to Spaces: models/gender_encoder.pkl
✅ Uploaded to Spaces: models/halfmarathon_model_latest.pkl
✅ Uploaded to Spaces: models/model_metadata_latest.pkl

🎉 Training pipeline completed successfully!


## 🧪 Test Predictions

In [10]:
# Test with sample data
test_cases = [
    {'Płeć_encoded': 1, 'Wiek': 30, '5 km Czas_seconds': 1200, '5 km Tempo': 4.0},
    {'Płeć_encoded': 0, 'Wiek': 25, '5 km Czas_seconds': 1500, '5 km Tempo': 5.0},
    {'Płeć_encoded': 1, 'Wiek': 45, '5 km Czas_seconds': 1350, '5 km Tempo': 4.5},
]

print("🧪 Test Predictions:\n")
for i, case in enumerate(test_cases, 1):
    # Create feature vector with proper column names
    test_df = pd.DataFrame([case])
    # Add missing columns with default values
    for col in feature_cols:
        if col not in test_df.columns:
            test_df[col] = 0
    test_df = test_df[feature_cols]
    
    prediction_seconds = best_model.predict(test_df)[0]
    prediction_time = f"{int(prediction_seconds//3600)}:{int((prediction_seconds%3600)//60):02d}:{int(prediction_seconds%60):02d}"
    
    gender = 'Mężczyzna' if case['Płeć_encoded'] == 1 else 'Kobieta'
    print(f"Test case {i}:")
    print(f"   {gender}, {case['Wiek']} lat, 5km: {case['5 km Czas_seconds']//60}:{case['5 km Czas_seconds']%60:02d}")
    print(f"   Predicted half-marathon time: {prediction_time}\n")

🧪 Test Predictions:

Test case 1:
   Mężczyzna, 30 lat, 5km: 20:00
   Predicted half-marathon time: 1:18:55

Test case 2:
   Kobieta, 25 lat, 5km: 25:00
   Predicted half-marathon time: 1:25:34

Test case 3:
   Mężczyzna, 45 lat, 5km: 22:30
   Predicted half-marathon time: 1:22:02

